In [11]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt


In [12]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta 

# Reflect Tables into SQLAlchemy ORM

In [13]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func , inspect
from sqlalchemy import desc

In [10]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite", echo = False)

In [11]:
# reflect an existing database into a new model
Homebase = automap_base()
# reflect the tables
Homebase.prepare(engine,reflect = True)

C:\Users\openbox\AppData\Local\Temp\ipykernel_7652\3844983111.py:4: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Homebase.prepare(engine,reflect = True)


In [12]:
# View all of the classes that automap found
Homebase.classes.keys()

['measurement', 'station']

In [13]:
# Save references to each table
Measurement = Homebase.classes.measurement
Station = Homebase.classes.station

In [14]:
# Create our session (link) from Python to the DB
session=Session(engine)

In [15]:
inspector = inspect(engine)

columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c['type'])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


# Exploratory Precipitation Analysis

In [16]:
# Find the most recent date in the data set.
most_recent_date = session.query(func.max(Measurement.date)).scalar()
print("The most recent date is ", most_recent_date)

The most recent date is  2017-08-23


In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.

twelve_months_ago = most_recent_date - timedelta(days=365)


# Perform a query to retrieve the data and precipitation scores
precipitation_data = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= twelve_months_ago).\
    filter(Measurement.date <= most_recent_date).\
    order_by(Measurement.date).all()
# print(precipitation_data)
# Save the query results as a Pandas DataFrame. Explicitly set the column names

dates = [result[0] for result in precipitation_data]
precipitation = [result[1] for result in precipitation_data]

# print(dates)
# print(precipitation)
# Sort the dataframe by date
combined_data = list(zip(dates, precipitation))
sorted_data = sorted(combined_data, key=lambda x: x[0])
sorted_dates, sorted_precipitation = zip(*sorted_data)
# print(sorted_dates)
# print(sorted_precipitation)
df = pd.DataFrame({'Date': sorted_dates, 'Precipitation': sorted_precipitation})

# # Use Pandas Plotting with Matplotlib to plot the data

# Plot the data using Pandas
df.plot(x='Date', y='Precipitation', figsize=(20, 10), kind='bar', legend=False)
plt.xlabel('Date')
plt.ylabel('Precipitation (inches)')
plt.title('Last 12 Months of Precipitation Data')

plt.tight_layout()
plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
station_num = session.query(func.count(Measurement.id)).all()
station_num

[(19550,)]

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
active_stations = session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
active_stations

[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
active_stations_desc = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs).filter(Measurement.station == 'USC00519281')).all()
active_stations_desc

[(53.0, 87.0, 71.66378066378067)]

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


active_station_data = session.query(Measurement.tobs).filter(Measurement.station == 'USC00519281'). filter(Measurement.date >= most_recent_date).all()
print(active_station_data)
#create df for active station annual results
active_station_data_df = pd.DataFrame(active_station_data,columns=['Tobs'])

#plot histogram for results
active_station_data_df.hist(bins=10)
plt.title("Annual Observed Temps for station 'USC00519281'", fontsize = 20)
plt.ylabel("Frequency" , fontsize = 10)
plt.xlabel("Temps")
plt.show()



[]


# Close Session

In [ ]:
# Close Session
session.close()